In [1]:
# -*- coding:utf-8 -*-
# author : apollo2mars@gmail.com
# function : use svm or other shallow model do imdb classification algorithm
# 使用CountVectorizer + n-gram提取特征
# 使用tfidf 获取特征权重
# 使用svm 等浅层方法进行分类

In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
# 添加路径读utils文件夹中的util文件
import sys
print(sys.path)
sys.path.append("/home/apollo/craft/Projects/Holy-Miner")
print(sys.path)
from utils.util import get_project_path

['', '/opt/anaconda3/lib/python36.zip', '/opt/anaconda3/lib/python3.6', '/opt/anaconda3/lib/python3.6/lib-dynload', '/opt/anaconda3/lib/python3.6/site-packages', '/opt/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/home/apollo/.ipython']
['', '/opt/anaconda3/lib/python36.zip', '/opt/anaconda3/lib/python3.6', '/opt/anaconda3/lib/python3.6/lib-dynload', '/opt/anaconda3/lib/python3.6/site-packages', '/opt/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/home/apollo/.ipython', '/home/apollo/craft/Projects/Holy-Miner']


ModuleNotFoundError: No module named 'utils'

In [2]:
root_path = get_project_path()
print(root_path)

NameError: name 'get_project_path' is not defined

In [5]:
def read_data_from_folder(folder_path):
    """
    read data from neg and pos folder
    """
    files = os.listdir(folder_path)
    all_line = []
    for file in files:
        if not os.path.isdir(file):
            with open(folder_path + '/' + file) as f:
                for line in f:
                    all_line.append(line)
    return  all_line

In [6]:
def get_data_feature_single(data):

    """
    TF-IDF
    """
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    # 最多特征数是1000
    vec = CountVectorizer(min_df=5, max_features=1000)  # 选择特征项
    transform = TfidfTransformer()  # 计算权值

    tfidf = transform.fit_transform(vec.fit_transform(data))
    tfidf_array = tfidf.toarray()
#     print(vec.get_feature_names())
#     print(tfidf_array)
    return tfidf_array

In [7]:
def get_data_feature_ngram(data):

    """
    TF-IDF
    """
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    # ngram 中的n 取值为2,3,4
    ngram_vectorizer = CountVectorizer(ngram_range=(2, 4), decode_error="ignore",
                                       token_pattern=r'\b\w+\b', min_df=5, max_features=100000) # 选择特征项
    transform = TfidfTransformer()  # 计算权值

    tfidf = transform.fit_transform(ngram_vectorizer.fit_transform(data))
#     print(ngram_vectorizer.get_feature_names())
#     print(tfidf.toarray())
    return tfidf

In [8]:
# read data
pos_data_folder = root_path + 'data/aclImdb/train/pos'
neg_data_folder = root_path + 'data/aclImdb/train/neg'
pos_text_data = read_data_from_folder(pos_data_folder)
neg_text_data = read_data_from_folder(neg_data_folder)

In [11]:
all_text_data = pos_text_data + neg_text_data
all_data = get_data_feature_ngram(all_text_data)

In [ ]:
print(all_data[0])

In [ ]:
print(all_data[1])

In [ ]:
print(all_data[2])

In [12]:
# get label
pos_label = np.ones(len(pos_text_data))
neg_label = np.zeros(len(neg_text_data))
all_label = list(pos_label) + list(neg_label)

print(" all data number is {}".format(all_data.shape[0]))
print(" all label number is {}".format(len(all_label)))

 all data number is 25000
 all label number is 25000


In [13]:
# merge pos and neg and split train and test
train_data, test_data, train_label, test_label = train_test_split(all_data, all_label, test_size=0.2, random_state=43)

In [14]:
print(all_data[0].shape)

(1, 100000)


In [15]:
#
# svm classification
#
param_C = 5
# param_gamma = 0.05
from sklearn import svm, metrics
svm_clf = svm.LinearSVC(C=param_C)
svm_clf.fit(train_data, train_label)
scores = svm_clf.score(test_data, test_label)
print(scores)

# predict
expected = test_label
predicted = svm_clf.predict(test_data)
print("Classification report for classifier %s:\n%s\n"
      % (svm_clf, metrics.classification_report(expected, predicted)))

cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)
print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


# 特征维度越大，精度越高

0.8858
Classification report for classifier LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0):
             precision    recall  f1-score   support

        0.0       0.89      0.88      0.88      2451
        1.0       0.88      0.89      0.89      2549

avg / total       0.89      0.89      0.89      5000


Confusion matrix:
[[2152  299]
 [ 272 2277]]
Accuracy=0.8858


In [16]:
#
# LR
#
from sklearn import linear_model
lr_clf = linear_model.LinearRegression()
lr_clf.fit(train_data, train_label)
scores = lr_clf.score(test_data, test_label)
print(scores)

0.530251132322


In [17]:
#
# DT
#
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(train_data, train_label)
scores = clf.score(test_data, test_label)
print(scores)

0.6634


In [18]:
#
# adaboost
#
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(train_data, train_label)
scores = clf.score(test_data, test_label)
print(scores)

0.731


In [24]:
#
# GBDT
#
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(train_data, train_label)
scores = clf.score(test_data, test_label)
print(scores)

0.7712


In [ ]:
#
# XGboost
#
import xgboost as xgb

In [20]:
#
# lightgbm
#
import lightgbm as lgb
lgb_train = lgb.Dataset(train_data, train_label)
lgb_eval = lgb.Dataset(test_data, test_label, reference=lgb_train)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

## predict
test_prd = gbm.predict(test_data)
print(test_prd[0])
print(test_label[0])
## accuracy
from sklearn.metrics import accuracy_score
# print(accuracy_score(test_label, test_prd))
from sklearn.metrics import roc_auc_score
print('The roc of prediction is:', roc_auc_score(test_label, test_prd) )

[1]	valid_0's auc: 0.715639	valid_0's l2: 0.245684
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.739664	valid_0's l2: 0.241427
[3]	valid_0's auc: 0.748341	valid_0's l2: 0.237355
[4]	valid_0's auc: 0.76497	valid_0's l2: 0.233343
[5]	valid_0's auc: 0.770186	valid_0's l2: 0.229987
[6]	valid_0's auc: 0.771421	valid_0's l2: 0.226876
[7]	valid_0's auc: 0.778657	valid_0's l2: 0.223613
[8]	valid_0's auc: 0.781384	valid_0's l2: 0.220908
[9]	valid_0's auc: 0.780733	valid_0's l2: 0.218534
[10]	valid_0's auc: 0.787273	valid_0's l2: 0.215853
[11]	valid_0's auc: 0.789556	valid_0's l2: 0.21365
[12]	valid_0's auc: 0.793751	valid_0's l2: 0.211384
[13]	valid_0's auc: 0.797258	valid_0's l2: 0.209345
[14]	valid_0's auc: 0.799425	valid_0's l2: 0.207256
[15]	valid_0's auc: 0.800201	valid_0's l2: 0.205642
[16]	valid_0's auc: 0.803767	valid_0's l2: 0.203619
[17]	valid_0's auc: 0.804644	valid_0's l2: 0.202314
[18]	valid_0's auc: 0.808054	valid_0's l2: 0.200469
[19]	valid_0's

In [25]:
# https://www.cnblogs.com/cnXuYang/p/8992865.html
'''Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
# Notes
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''

# pos_text_data = read_data_from_folder(pos_data_folder)
# neg_text_data = read_data_from_folder(neg_data_folder)

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32


print("Preprocess...")
t = Tokenizer()
t.fit_on_texts(pos_text_data + neg_text_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(pos_text_data + neg_text_data)
# pad documents to a max length of 4 words
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

train_data, test_data, train_label, test_label = train_test_split(padded_docs, all_label, test_size=0.2, random_state=42)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print('Train...')
model.fit(train_data, train_label, batch_size=batch_size, epochs=15, validation_data=(test_data, test_label))
score, acc = model.evaluate(test_data, test_label, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Preprocess...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
Train...


ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 20000 arrays: [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0,...